In [1]:
import torch
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import Subset, DataLoader
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import datasets
import math
# Required libraries are imported

In [2]:
!pip install --upgrade wandb
import wandb
# import socket
# socket.setdefaulttimeout(30)
wandb.login(key='1d2423ec9b728fe6cc1e2c0b9a2af0e67a45183c')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 69.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: wandb
    Found existing installation: wandb 0.19.6
    Uninstalling wandb-0.19.6:
      Successfully uninstalled wandb-0.19.6


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs24m047 (cs24m047-iitm-ac-in) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
train_directory='/kaggle/input/dataset2/inaturalist_12K/train'
test_directory='/kaggle/input/dataset2/inaturalist_12K/val'

In [5]:
class CNN(nn.Module):
    def __init__(self, no_of_input_channels=3, no_of_classes=10,no_of_filters=[32,32,32,32,32],size_of_filter=[3,3,3,3,3],no_of_neurons=128,
                            activation_function='sigmoid',dropout_probability=0.0,batch_normalization='no'):
        super(CNN, self).__init__()
        self.activation_function_name = activation_function
        self.batch_normalization = batch_normalization
        # if(filter_organizing_mode ==  0): # the filters are same
        #     no_of_filters = [no_of_filters_in_base,no_of_filters_in_base,no_of_filters_in_base,no_of_filters_in_base,no_of_filters_in_base]
        # else:
        #     no_of_filters = [no_of_filters_in_base,no_of_filters_in_base*2,no_of_filters_in_base*4,no_of_filters_in_base*8,no_of_filters_in_base*16] # the filters increase by *2
        width = 0.0
        height = 0.0

        self.conv_layer1 = nn.Conv2d(in_channels=no_of_input_channels,out_channels=no_of_filters[0],kernel_size=size_of_filter[0], stride=1)
        width = (256 - size_of_filter[0])+1 #256 --> width of image
        height = (256 - size_of_filter[0])+1 #256 --> height of image
        self.batch_norm1 = nn.BatchNorm2d(no_of_filters[0])  # batch normalizations
        self.pool_layer1 = nn.MaxPool2d(kernel_size=size_of_filter[0], stride=2)
        width = math.floor((width - size_of_filter[0])/2) + 1 # width  calculatios of feature map
        height = math.floor((height -size_of_filter[0])/2) + 1 # hight calculations of feature map

        self.conv_layer2 = nn.Conv2d( in_channels=no_of_filters[0], out_channels=no_of_filters[1], kernel_size=size_of_filter[1], stride=1)
        width = ((width - size_of_filter[1]))+1
        height = ((height-size_of_filter[1]))+1
        self.batch_norm2 = nn.BatchNorm2d(no_of_filters[1])  # batch normalizations
        self.pool_layer2 = nn.MaxPool2d(kernel_size=size_of_filter[1], stride=2)
        width = math.floor((width - size_of_filter[1])/2) + 1 # width  calculatios of feature map
        height = math.floor((height -size_of_filter[1])/2) + 1 # hight calculations of feature map

        self.conv_layer3 = nn.Conv2d( in_channels=no_of_filters[1], out_channels=no_of_filters[2], kernel_size=size_of_filter[2], stride=1)
        width = ((width - size_of_filter[2]))+1
        height = ((height-size_of_filter[2]))+1
        self.batch_norm3 = nn.BatchNorm2d(no_of_filters[2])  # batch normalizations
        self.pool_layer3 = nn.MaxPool2d(kernel_size=size_of_filter[2], stride=2)
        width = math.floor((width - size_of_filter[2])/2) + 1 # width  calculatios of feature map
        height = math.floor((height -size_of_filter[2])/2) + 1 # hight calculations of feature map

        self.conv_layer4 = nn.Conv2d( in_channels=no_of_filters[2], out_channels=no_of_filters[3], kernel_size=size_of_filter[3], stride=1)
        width = ((width - size_of_filter[3]))+1
        height = ((height-size_of_filter[3]))+1
        self.batch_norm4 = nn.BatchNorm2d(no_of_filters[3])  # batch normalizations
        self.pool_layer4 = nn.MaxPool2d(kernel_size=size_of_filter[3], stride=2)
        width = math.floor((width - size_of_filter[3])/2) + 1 # width  calculatios of feature map
        height = math.floor((height -size_of_filter[3])/2) + 1 # hight calculations of feature map

        self.conv_layer5 = nn.Conv2d( in_channels=no_of_filters[3], out_channels=no_of_filters[4], kernel_size=size_of_filter[4], stride=1)
        width = ((width - size_of_filter[4]))+1
        height = ((height-size_of_filter[4]))+1
        self.batch_norm5 = nn.BatchNorm2d(no_of_filters[4])  # batch normalizations
        self.pool_layer5 = nn.MaxPool2d(kernel_size=size_of_filter[4], stride=2)
        width = math.floor((width - size_of_filter[4])/2) + 1 # width  calculatios of feature map
        height = math.floor((height -size_of_filter[4])/2) + 1 # hight calculations of feature map

        self.dropout = nn.Dropout(p=dropout_probability) # added dropout to overcome overfitting.
        self.full_connected1 = nn.Linear(no_of_filters[4] * width * height, no_of_neurons)
        self.batch_norm6 = nn.BatchNorm1d(no_of_neurons)
        self.full_connected2 = nn.Linear(no_of_neurons, no_of_classes)

    def forward(self, x):
        if(self.activation_function_name == 'relu'):
            activation_function = F.relu
        elif(self.activation_function_name == 'gelu'):
            activation_function = F.gelu
        elif(self.activation_function_name == 'silu'):
            activation_function = F.silu
        else:
            activation_function = F.mish

        if(self.batch_normalization == 'yes'):
            x = activation_function(self.batch_norm1(self.conv_layer1(x)))
        else:
            x = activation_function(self.conv_layer1(x))
        x = self.pool_layer1(x)

        if(self.batch_normalization == 'yes'):
            x = activation_function(self.batch_norm2(self.conv_layer2(x)))
        else:
            x = activation_function(self.conv_layer2(x))
        x = self.pool_layer2(x)

        if(self.batch_normalization == 'yes'):
            x = activation_function(self.batch_norm3(self.conv_layer3(x)))
        else:
            x = activation_function(self.conv_layer3(x))
        x = self.pool_layer3(x)

        if(self.batch_normalization == 'yes'):
            x = activation_function(self.batch_norm4(self.conv_layer4(x)))
        else:
            x = activation_function(self.conv_layer4(x))
        x = self.pool_layer4(x)

        if(self.batch_normalization == 'yes'):
            x = activation_function(self.batch_norm5(self.conv_layer5(x)))
        else:
            x = activation_function(self.conv_layer5(x))
        x = self.pool_layer5(x)

        x = x.reshape(x.shape[0], -1)
        if(self.batch_normalization == 'yes'):
            x = activation_function(self.batch_norm6(self.full_connected1(x)))
        else:
            x = activation_function(self.full_connected1(x))
        x = self.dropout(x)
        x = self.full_connected2(x)
        return x

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
transform_basic = transforms.Compose([
    transforms.Resize((256,256)), # resized to a threshold value so that all images have same shape and size
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))]) # normalized for better accuracy.

train_dataset = datasets.ImageFolder(root=train_directory,transform=transform_basic) # train_data loading
training_dataset,validation_dataset = torch.utils.data.random_split(train_dataset,[8000,1999]) #splitting the data into 80%(training) and 20%(validation) The overall data size is 9999

transform_augmented = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(10),      # Randomly rotate the image by a maximum of 10 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust brightness, contrast, saturation, and hue
    transforms.RandomResizedCrop(256),  # Randomly crop and resize the image to 256x256
    transforms.ToTensor(),              # Convert the image to a PyTorch tensor
    transforms.Normalize((0.5,),(0.5,))  # Normalize the image
]) # for augumenting the training data
train_dataset2 = datasets.ImageFolder(root=train_directory,transform=transform_augmented)
training_dataset_aug,validation_dataset_aug = torch.utils.data.random_split(train_dataset2,[8000,1999]) #  #splitting the data into 80%(training) and 20%(validation) The overall data size is 9999

test_dataset = datasets.ImageFolder(root=test_directory,transform=transform_basic); # test data loading.

In [8]:
def data_loader_creator(augmentation_flag,batch_size): # function to return the data loaders depending on augumentation.
    if(augmentation_flag == 'no'):
        train_loader = torch.utils.data.DataLoader(training_dataset,batch_size =batch_size,shuffle = True,num_workers=2,pin_memory=True)
        val_loader = torch.utils.data.DataLoader(validation_dataset,batch_size =batch_size,shuffle = True,num_workers=2,pin_memory=True)
        return train_loader,val_loader
    else:
        train_loader_aug = torch.utils.data.DataLoader(training_dataset_aug,batch_size =batch_size,shuffle = True,num_workers=4,pin_memory=True)
        val_loader_aug = torch.utils.data.DataLoader(validation_dataset_aug,batch_size =batch_size,shuffle = True,num_workers=4,pin_memory=True)
        return train_loader_aug,val_loader_aug

In [9]:
def Accuracy_calculator(loader,model,criterion,batch_size): # function to clculate the accuracy and loss
    no_of_correct_predictions = 0
    no_of_samples = 0
    total_loss = 0.0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            scores = model(x)
            loss = criterion(scores, y)
            total_loss += loss.item()*batch_size # sum of cross entropies
            _, predictions = scores.max(1)
            no_of_correct_predictions += (predictions == y).sum().item() # correctly classified data
            no_of_samples += predictions.size(0)
    model.train()
    return (no_of_correct_predictions / no_of_samples)*100 , total_loss / no_of_samples

In [10]:
def train_the_model(no_of_neurons,no_of_filters,size_of_filter,activation_function_name,optimizer_name,batch_size,
                    dropout_probability,no_of_epochs,learning_rate,batch_normalization,augmentation_flag):

    train_loader,val_loader = data_loader_creator(augmentation_flag,batch_size)  # getting dataloaders.

    #test_loader = torch.utils.data.DataLoader(test_data,batch_size =batchSize,shuffle = True,num_workers=2,pin_memory=True)

    no_of_input_channels=3
    no_of_classes=10

    model=CNN(no_of_input_channels, no_of_classes,no_of_filters,size_of_filter,no_of_neurons,
              activation_function_name,dropout_probability,batch_normalization).to(device)
    # model=nn.DataParallel(model)
    # model=model.to(device)

    if(optimizer_name == 'sgd'):
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    elif(optimizer_name == 'adam'):
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    else:
      optimizer = optim.NAdam(model.parameters(), lr=learning_rate) # optimzers selection

    criterion = nn.CrossEntropyLoss() # since it is classification problem corss entropy loss is used.

    for epoch in range(no_of_epochs): # performs the training.
        for batchId, (input_images, target_classes) in enumerate(tqdm(train_loader)):
            # Get data to cuda if possible
            input_images = input_images.to(device=device)
            target_classes = target_classes.to(device=device)
            # forward
            scores = model(input_images) # give the last layer pre-activation values.
            loss = criterion(scores,target_classes) # gets the overll cross entropy loss for each batch
            optimizer.zero_grad() # gradients are made to zero for each batch.
            loss.backward()  # calculaing the gradients
            optimizer.step() #updates the parameters
        training_accuracy,training_loss = Accuracy_calculator(train_loader, model,criterion,batch_size) # calculates the accuracy and loss at one go
        validation_accuracy,validation_loss = Accuracy_calculator(val_loader, model,criterion,batch_size)
        #  the below line can be uncommenteed for test accuracy and loss
        #test_accuracy,test_loss = check_accuracy(test_loader, model,criterion,batchSize)
        print(f"training_accuracy:{training_accuracy:.4f},training_loss:{training_loss:.4f}")
        print(f"validation_accuracy:{validation_accuracy:.4f},validation_loss:{validation_loss:.4f}")
        #print(f"test_accuracy:{test_accuracy:.4f},test_loss:{test_loss:.4f}")
        wandb.log({'training_accuracy':training_accuracy}) # plotting  the data in wandb
        wandb.log({'training_loss':training_loss})
        wandb.log({'validation_accuracy':validation_accuracy})
        wandb.log({'validation_loss':validation_loss})

In [11]:
# Sweep config for wandb plotting
# wandb.init(project ='DA6401_Assignment_2')
sweep_config = {
    'name'  : "run_count_20",
    'method': 'bayes',
    'metric': {
      'name': 'validation_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'no_of_neurons': {
            'values': [128, 256, 512]
        },
        'no_of_filters': {
            'values': [[64,128,256,512, 1024], [32,32,32,32,32],[32,64,64,128,128],[128,128,64,64,32],[32,64,128,256,512]]
        },
        'size_of_filter': {
            'values': [[3,3,3,3,3], [5,5,5,5,5], [5,3,5,3,5]]
        },
        'activation_function_name': {
            'values': ['relu','gelu','silu','mish']
        },
        'optimizer_name': {
            'values': ['nadam', 'adam']
        },
        'batch_size': {
            'values': [32, 64,128]
        },
        'dropout_probability': {
            'values': [0, 0.2, 0.4]
        },
        'no_of_epochs': {
            'values': [5,10]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4]
        },
        'batch_normalization': {
            'values': ['yes','no']
        },
        'augmentation_flag': {
            'values': ['yes','no']
        }
    }
}

# sweep_id = wandb.sweep(sweep_config, project="DA6401_Assignment_2")

In [ ]:
def run_experiment():
    try:
        run = wandb.init()  # No config argument here
        cfg = run.config
        run.name = (
            f"No_of_neurons: {cfg.no_of_neurons}, "
            f"No_of_filters: {cfg.no_of_filters}, "
            f"Size_of_filter: {cfg.size_of_filter}, "
            f"Activation_function: {cfg.activation_function_name}, "
            f"Optimizer: {cfg.optimizer_name}, "
            f"Batch_size: {cfg.batch_size}, "
            f"Dropout: {cfg.dropout_probability}, "
            f"No_of_epochs: {cfg.no_of_epochs}, "
            f"Learning_Rate: {cfg.learning_rate}, "
            f"Batch_normalization: {cfg.batch_normalization}, "
            f"Augmentation_flag: {cfg.augmentation_flag}"
        )
        train_the_model(
            cfg.no_of_neurons,
            cfg.no_of_filters,
            cfg.size_of_filter,
            cfg.activation_function_name,
            cfg.optimizer_name,
            cfg.batch_size,
            cfg.dropout_probability,
            cfg.no_of_epochs,
            cfg.learning_rate,
            cfg.batch_normalization,
            cfg.augmentation_flag
        )
    except Exception as e:
        print(f"Error during training: {e}")
        if wandb.run:
            wandb.finish(exit_code=1)
        raise
    finally:
        if wandb.run:
            wandb.finish
if __name__=="__main__":
    sweep_id = wandb.sweep(sweep_config, project="DA6401_Assignment_2")
    wandb.agent(sweep_id, run_experiment ,  count=20)

Create sweep with ID: nnr2n1o8
Sweep URL: https://wandb.ai/cs24m047-iitm-ac-in/DA6401_Assignment_2/sweeps/nnr2n1o8


wandb: Agent Starting Run: zeks2cy7 with config:
wandb: 	activation_function_name: silu
wandb: 	augmentation_flag: no
wandb: 	batch_normalization: no
wandb: 	batch_size: 64
wandb: 	dropout_probability: 0.2
wandb: 	learning_rate: 0.001
wandb: 	no_of_epochs: 5
wandb: 	no_of_filters: [128, 128, 64, 64, 32]
wandb: 	no_of_neurons: 128
wandb: 	optimizer_name: adam
wandb: 	size_of_filter: [3, 3, 3, 3, 3]


100%|██████████| 125/125 [01:27<00:00,  1.44it/s]
